In [0]:
FLUXSTD_FILE = f'/datascope/subaru/data/targeting/dSph/draco/fluxstd_draco_miho_20250315.feather'
SKY_FILE = f'/datascope/subaru/data/targeting/dSph/draco/sky_draco_miho_20250315.feather'
OBS_PATH = f'/datascope/subaru/data/targeting/dSph/draco/draco_tpall3e_g24.cat'

GALAXY = 'draco'

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

In [0]:
# if 'debug' not in globals():
#     import debugpy
#     debugpy.listen(('0.0.0.0', 5698))
#     debug = True

# Plot a flux standard file

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color

In [0]:
from pfs.ga.targeting.io import PfsFluxStdReader, PfsSkyReader
from pfs.ga.targeting.data import Observation

In [0]:
fluxstd = PfsFluxStdReader().read(FLUXSTD_FILE)
sky = PfsSkyReader().read(SKY_FILE)

In [0]:
fluxstd.data.columns

In [0]:
if 'prob_f_star' in fluxstd.data.columns:
    plt.hist(fluxstd.data['prob_f_star'].values)

In [0]:
from pfs.ga.targeting.util.astro import *

In [0]:
Jy_to_ABmag(fluxstd.data['psf_flux_g'])

In [0]:
mask = fluxstd.data['prob_f_star'] > 0.9

plt.plot(
    Jy_to_ABmag(fluxstd.data['psf_flux_g'][mask]) - Jy_to_ABmag(fluxstd.data['psf_flux_r'][mask]),
    Jy_to_ABmag(fluxstd.data['psf_flux_g'][mask]), '.', ms=0.5)

plt.gca().invert_yaxis()

In [0]:
# Check if any of the columns contain None values

for c in fluxstd.data.columns:
    none = fluxstd.data[c].isna().sum()
    print(c, none)

# np.any(fluxstd.data['psf_mag_g'].isna())

In [0]:
sky.data.columns

In [0]:
obs = SubaruHSC.text_observation_reader().read(OBS_PATH)
obs.data.head()

# Plot field of view of instrument

In [0]:
from pfs.ga.targeting.targets.dsph import GALAXIES
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.targeting.diagram import FOV, FP

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle

In [0]:
target = GALAXIES[GALAXY]

In [0]:
pointing = target.get_pointings(SubaruPFI)[0]
print(pointing.ra, pointing.dec, pointing.posang)

wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
pfi = SubaruPFI(projection=wfc, instrument_options={ 'layout': 'calibration' })
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=120, subplot_kw=dict(projection=fov.projection.wcs))

# fov.plot_catalog(ax, sky, color='b', alpha=0.3)
fov.plot_catalog(ax, fluxstd, color='r', size=2)
# fov.plot_catalog(ax, obs, color='k')

# ax.plot(target.pos.ra, target.pos.dec, marker='x', markersize=30, color='r', transform=ax.get_transform('world'))

pfi.plot_focal_plane(ax, fov, color='red', lw=1, corners=True)

ax.set_xlim(2, -3)
ax.set_ylim(-3, 2)

ax.grid()

In [0]:
from pfs.ga.targeting.util.astro import *

In [0]:
# Calculate the magnitudes from the flux
fluxstd.data['psf_mag_r'] =  nJy_to_ABmag(fluxstd.data['psf_flux_r'])
fluxstd.data['psf_mag_i'] =  nJy_to_ABmag(fluxstd.data['psf_flux_i'])
fluxstd.data['psf_mag_z'] =  nJy_to_ABmag(fluxstd.data['psf_flux_z'])

In [0]:
plt.plot(fluxstd.data['psf_mag_g'] - fluxstd.data['psf_mag_r'], fluxstd.data['psf_mag_g'], '.')
plt.gca().invert_yaxis()
plt.xlabel('PS g - r')
plt.ylabel('PS g')